<a href="https://colab.research.google.com/github/tayyabrafi/Strategy/blob/main/Fetching%20Candles%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import requests
import typing
import pandas as pd

public_key = '18fcff7e215444a925f2a24335876c8848b093c767be3ada2a4fa961e7120a1c'
base_url = 'https://testnet.binancefuture.com'
headers = {'X-MBX-APIKEY': public_key}


class Candle:
    def __init__(self, candle_info, exchange: str, timeframe: str):
        if exchange == "binance":
            self.timestamp = candle_info[0]
            self.open = float(candle_info[1])
            self.high = float(candle_info[2])
            self.low = float(candle_info[3])
            self.close = float(candle_info[4])
            self.volume = float(candle_info[5])

In [17]:

def make_request(method: str, endpoint: str, data: typing.Dict):
    if method == 'GET':
        try:
            response = requests.get(base_url + endpoint, params=data, headers=headers)
        except Exception as e:
            print("Connection error while making %s request to %s: %s", method, endpoint, e)
            return None

    elif method == 'POST':
        try:
            response = requests.post(base_url + endpoint, params=data, headers=self.headers)
        except Exception as e:
            print("Connection error while making %s request to %s: %s", method, endpoint, e)
            return None

    elif method == 'DELETE':
        try:
            response = requests.delete(base_url + endpoint, params=data, headers=self.headers)
        except Exception as e:
            print("Connection error while making %s request to %s: %s", method, endpoint, e)
            return None
    else:
        raise ValueError()

    if response.status_code == 200:
        return response.json()
    else:
        print("Error while making %s request to %s: %s (error code %s)",
                      method, endpoint, response.json(), response.status_code)

def get_historical_candles(symbol: str, interval: str) -> typing.List[Candle]:

  data = dict()
  data['symbol'] = symbol
  data['interval'] = interval
  data['limit'] = 1500

  raw_candles = make_request('GET', '/fapi/v1/klines', data)

  candles = []

  if raw_candles is not None:
      for c in raw_candles:

          candles.append(Candle(c, "binance", interval))

  # candles is a list of objects of class Candle which contains(high, low, volume, etc.)
  # each object has a candle containing values of high, low, open, close of that particular timeframe
  return candles




In [18]:
candles = get_historical_candles('BTCUSDT','1h')
timestamp = []
open = []
high =[]
low = []
close = []
volume = []

for candle in candles:
    timestamp.append(candle.timestamp)
    open.append(candle.open)
    high.append(candle.high)
    low.append(candle.low)
    close.append(candle.close)
    volume.append(candle.volume)

data = {'timestamp':timestamp , 'open':open, 'high':high, 'low':low, 'close':close, 'volume':volume}
df = pd.DataFrame(data, index=data['timestamp'])
df.set_index('timestamp', inplace=True)


In [19]:
df

,open,high,low,close,volume
timestamp,,,,,
1690322400000,29205.5,29360.0,29124.0,29297.8,3757.365
1690326000000,29290.3,29398.1,29124.0,29216.5,4026.100
1690329600000,29217.9,29386.9,29164.0,29226.3,4400.769
1690333200000,29208.1,29391.3,29108.3,29158.8,3851.735
1690336800000,29224.8,29421.6,29108.3,29380.0,3519.524
...,...,...,...,...,...
1695704400000,26402.6,26474.1,26269.0,26269.0,4074.523
1695708000000,26334.3,26472.0,26242.4,26386.5,3958.057
1695711600000,26308.2,26469.2,26179.6,26353.3,3382.536
